In [2]:
# Проверка типов данных в DataFrame
print(df.dtypes)


PassengerId      int64
Survived         int64
Pclass           int64
Name            object
Sex              int64
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Cabin           object
Embarked        object
dtype: object


In [3]:
# Заполнение пропущенных значений только в числовых столбцах
numeric_cols = df.select_dtypes(include=['number']).columns
df[numeric_cols] = df[numeric_cols].fillna(df[numeric_cols].median())


In [5]:
# Преобразование категориальных данных в числовые
df['Sex'] = df['Sex'].map({'male': 0, 'female': 1})


In [7]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_curve, auc, classification_report
import matplotlib.pyplot as plt

# Загрузка данных
df = pd.read_csv('../data/raw/train.csv')

# Преобразование категориальных данных
df['Sex'] = df['Sex'].map({'male': 0, 'female': 1})

# Обработка пропущенных значений
# Заполнение пропусков только в числовых столбцах
numeric_cols = df.select_dtypes(include=['number']).columns
df[numeric_cols] = df[numeric_cols].fillna(df[numeric_cols].median())

# Определение признаков и целевой переменной
X = df[['Pclass', 'Sex', 'Age', 'Fare']]
y = df['Survived']

# Разделение данных на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Обучение модели
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

# Оценка модели
y_prob = model.predict_proba(X_test)[:, 1]
fpr, tpr, _ = roc_curve(y_test, y_prob)
roc_auc = auc(fpr, tpr)

plt.figure(figsize=(10, 6))
plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic')
plt.legend(loc='lower right')
plt.savefig('../reports/figures/roc_curve.png')
plt.close()

# Вывод классификационного отчета
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))

# Сохранение модели
import joblib
joblib.dump(model, '../models/model.pkl')


              precision    recall  f1-score   support

           0       0.82      0.86      0.84       105
           1       0.78      0.73      0.76        74

    accuracy                           0.80       179
   macro avg       0.80      0.79      0.80       179
weighted avg       0.80      0.80      0.80       179



['../models/model.pkl']